# Nonparametric methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import plot_tree
sns.set()

In [2]:
#Logistic regression on the mroz dataset
mroz = pd.read_csv("MROZ.csv")
print(mroz.shape)
mroz.head(12)

(753, 22)


,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300,0.6215,12,14,9.5,1,7,20.100058,1.524272,49
5,1,2032,0,0,54,12,4.7421,4.70,1040,57,...,19495,0.6915,14,7,7.5,1,33,9.859054,1.556480,1089
6,1,1440,0,2,37,16,8.3333,5.95,2670,37,...,21152,0.6915,14,7,5.0,0,11,9.152048,2.120260,121
7,1,1020,0,0,54,12,7.8431,9.98,4120,53,...,18900,0.6915,3,3,5.0,0,35,10.900038,2.059634,1225
8,1,1458,0,2,48,12,2.1262,0.00,1995,52,...,20405,0.7515,7,7,3.0,0,24,17.305000,0.754336,576
9,1,1600,0,2,39,12,4.6875,4.15,2100,43,...,20425,0.6915,7,7,5.0,0,21,12.925000,1.544899,441


In [ ]:
Y = mroz['inlf'] #this is a binary variable
X = mroz[['kidslt6', 'age', 'educ', 
        'huswage', 'exper', 'expersq']].copy()
X = sm.add_constant(X)
logimodel = sm.GLM(Y, X, family=sm.families.Binomial()).fit()
print(logimodel.summary())

What is the interpretation of the coefficient -1.4516 of the 'kidslt6' variable? It means that having a small kid reduces the log-odds of working by -1.4516. This means that the odds are reduced by a factor of $\exp(-1.4516) \approx 0.234$. 

In [ ]:
#We used statsmodels.api for fitting GLMs. Today we shall switch to scikit-learn because it also fits nonparametric models such as decision trees and random forests
#Let us re-fit this logistic regression model using scikit-learn
Y = mroz['inlf']  # This is a binary variable
X = mroz[['kidslt6', 'age', 'educ', 'huswage', 'exper', 'expersq']].copy()
from sklearn.linear_model import LogisticRegression
logimodel_sk = LogisticRegression(max_iter = 1000)
logimodel_sk.fit(X, Y)
print("Intercept:", logimodel_sk.intercept_)
print("Coefficients:", logimodel_sk.coef_)

These coefficients are slightly different from those given by statsmodels.api because scikit-learn does not fit the MLE exactly but also imposes some L2 regularization (similar to ridge regression). The usual MLE can be obtained by setting penalty = None as shown below. 

In [ ]:
Y = mroz['inlf']  # This is a binary variable
X = mroz[['kidslt6', 'age', 'educ', 'huswage', 'exper', 'expersq']].copy()
from sklearn.linear_model import LogisticRegression
logimodel_sk = LogisticRegression(penalty=None, max_iter = 1000)
logimodel_sk.fit(X, Y)
print("Intercept:", logimodel_sk.intercept_)
print("Coefficients:", logimodel_sk.coef_)

In [ ]:
#Accuracy of the logistic regression model:
#Split the original data into training and test datasets
#We will fit the model on the training dataset and test its accuracy on the test dataset:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=503, test_size=250, random_state=1)

In [ ]:
#Let us fit logistic regression on the training dataset:
logimodel_train = LogisticRegression(penalty = None, max_iter = 1000)
logimodel_train.fit(X_train, Y_train)
print("Intercept:", logimodel_train.intercept_)
print("Coefficients:", logimodel_train.coef_)

In [ ]:
#Prediction for the test data
Y_pred = logimodel_train.predict(X_test)
print(Y_pred)
accuracy_logimodel = np.mean(Y_test == Y_pred)
print(f"Accuracy of Logistic Regression on test set: {accuracy_logimodel}")

In [ ]:
contingency_table = pd.crosstab(Y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(contingency_table)

In [ ]:
#Using k-NN classification:
k_value = 8
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k_value)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test)
accuracy_knn = np.mean(Y_test == Y_pred_knn)
print(f"Accuracy of k-NN Classification on test set: {accuracy_knn}")
#Contingency table (or confusion matrix)
print(pd.crosstab(Y_test, Y_pred_knn, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize and train the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, Y_train)

# Predict on the test set
Y_pred_dt = dt.predict(X_test)

# Calculate the accuracy
accuracy_dt = np.mean(Y_test == Y_pred_dt)
print(f"Accuracy of Decision Tree Classification on test set: {accuracy_dt}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_dt, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [ ]:
# Plot the tree
from sklearn.tree import plot_tree

plt.figure(figsize=(20,10))
plot_tree(dt, feature_names=X.columns,class_names = ('0','1'),  filled=True, rounded=True)
plt.show()

In [ ]:
#This tree is way too big to allow any meaningful interpretation
#Here are a few ways to create smaller trees:
dt_small = DecisionTreeClassifier(max_depth = 3, random_state = 1)
dt_small.fit(X_train, Y_train)

Y_pred_dt_small = dt_small.predict(X_test)
# Calculate the accuracy
accuracy_dt_small = np.mean(Y_test == Y_pred_dt_small)
print(f"Accuracy of Decision Tree Classification on test set: {accuracy_dt_small}")

# Plot the tree
from sklearn.tree import plot_tree

plt.figure(figsize=(20,10))
plot_tree(dt_small, feature_names=X.columns, class_names = ('0','1'),  filled=True, rounded=True)
plt.show()

In [ ]:
#Random Forest:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest classifier
rf = RandomForestClassifier(random_state=1, max_features = None)
rf.fit(X_train, Y_train)

# Predict on the test set
Y_pred_rf = rf.predict(X_test)

# Calculate the accuracy
accuracy_rf = np.mean(Y_test == Y_pred_rf)
print(f"Accuracy of Random Forest Classification on test set: {accuracy_rf}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_rf, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [ ]:
#The Random Forest Algorithm has many hyperparameters that one can hope to tune to achieve better accuracy in practice
#help(RandomForestClassifier)

## Some Simulated Toy Datasets

In [ ]:
#EXAMPLE ONE:
n = 750
n_train = 500
n_test = 250

x1 = np.random.uniform(-1, 1, n)
x2 = np.random.uniform(-1, 1, n)
X = np.vstack([x1, x2]).transpose()

Y = (x1 * x2 > 0).astype(np.int64)

#Split the dataset into training and test datasets of sizes 500 and 250 respectively
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=500, test_size=250, random_state=1)

x1_train = X_train[:,0]
x2_train = X_train[:,1]
x1_test = X_test[:,0]
x2_test = X_test[:,1]


In [ ]:
#In this case where there are only two features, the data can be plotted 
#color of the points denotes Y
def draw_results(x1, x2, color, plot_title=''):
    plt.figure()
    plt.scatter(x1, x2, c=color, cmap='viridis', alpha=0.7);
    plt.colorbar()
    plt.title(plot_title)
    plt.axis('equal')
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.tight_layout()

draw_results(x1_train, x2_train, color=Y_train, plot_title='Training data')
draw_results(x1_test, x2_test, color=Y_test, plot_title='Test data (ground truth)')

In [ ]:
#Method One: Logistic regression:
logimodel_train = LogisticRegression(penalty = None, max_iter = 1000)
logimodel_train.fit(X_train, Y_train)
print("Intercept:", logimodel_train.intercept_)
print("Coefficients:", logimodel_train.coef_)
#Prediction for the test data
Y_pred = logimodel_train.predict(X_test)
print(Y_pred)
accuracy_logimodel = np.mean(Y_test == Y_pred)
print(f"Accuracy of Logistic Regression on test set: {accuracy_logimodel}")

In [ ]:
# Visualize the results
pred_probs_logistic = logimodel_train.predict_proba(X_test)[:,1]
draw_results(
    x1_test, x2_test, color=pred_probs_logistic, 
    plot_title="Predicted probability of y=1 (logistic)"
)

draw_results(
    x1_test, x2_test, color=Y_pred, 
    plot_title="Logistic model prediction"
)

In [ ]:
#Logistic Regression will work here if we do feature engineering (include x1 * x2 as a third feature):
# Create a new feature: x1 * x2
#the following function returns an array like X but with a new feature that is x1 * x2
def add_mult_feature(X):
    """Returns an array like X, but with a new feature that's X1 * X2"""
    new_feature = X[:, 0] * X[:, 1]
    return np.hstack([X, new_feature[:, None]])

X_train_feat = add_mult_feature(X_train)
X_test_feat = add_mult_feature(X_test)

logimodel_train_feat = LogisticRegression(penalty = None, max_iter = 2000)
logimodel_train_feat.fit(X_train_feat, Y_train)
print("Intercept:", logimodel_train_feat.intercept_)
print("Coefficients:", logimodel_train_feat.coef_)
#Prediction for the test data
Y_pred_feat = logimodel_train_feat.predict(X_test_feat)
print(Y_pred_feat)
accuracy_logimodel_feat = np.mean(Y_test == Y_pred_feat)
print(f"Accuracy of Logistic Regression (with new feature x1*x2) on test set: {accuracy_logimodel_feat}")

pred_probs_logistic_feat = logimodel_train_feat.predict_proba(X_test_feat)[:,1]
draw_results(
    x1_test, x2_test, color=pred_probs_logistic_feat, 
    plot_title="Predicted probability of y=1 (logistic)"
)

draw_results(
    x1_test, x2_test, color=Y_pred_feat, 
    plot_title="Logistic model prediction"
)

In [ ]:
#Method Two: k-NN
#Using k-NN classification:
k_value = 8
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k_value)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test)
accuracy_knn = np.mean(Y_test == Y_pred_knn)
print(f"Accuracy of k-NN Classification on test set: {accuracy_knn}")
#Contingency table (or confusion matrix)
print(pd.crosstab(Y_test, Y_pred_knn, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [ ]:
#Visualizing the results:
from sklearn.neighbors import KNeighborsClassifier

probs_knn = knn.predict_proba(X_test)[:, 1]
y_hat_knn = (probs_knn > 0.5).astype(np.int64)


draw_results(
    x1_test, x2_test, color=probs_knn, 
    plot_title="Predicted probability of y=1 (k-NN)"
)

draw_results(
    x1_test, x2_test, color=y_hat_knn, 
    plot_title="k-NN Model prediction"
)


In [ ]:
#Model Three: Decision Tree
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, Y_train)

# Predict on the test set
Y_pred_dt = dt.predict(X_test)

# Calculate the accuracy
accuracy_dt = np.mean(Y_test == Y_pred_dt)
print(f"Accuracy of Decision Tree Classification on test set: {accuracy_dt}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_dt, rownames=['Actual'], colnames=['Predicted'], margins=True))


In [ ]:
# Plot the tree
from sklearn.tree import plot_tree
plt.figure(figsize=(20,10))
plot_tree(dt, feature_names=['x1', 'x2'],class_names = ('0','1'),  filled=True, rounded=True)
plt.show()

In [ ]:
#Model Four: Random Forest
rf = RandomForestClassifier(random_state=1, max_features = None)
rf.fit(X_train, Y_train)

# Predict on the test set
Y_pred_rf = rf.predict(X_test)

# Calculate the accuracy
accuracy_rf = np.mean(Y_test == Y_pred_rf)
print(f"Accuracy of Random Forest Classification on test set: {accuracy_rf}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_rf, rownames=['Actual'], colnames=['Predicted'], margins=True))


In [ ]:
#EXAMPLE TWO: Let us add some training noise to the previous dataset:
Y_train_noisy = Y_train.copy()

pts_to_flip = np.random.random(n_train) < 0.1 #we are selecting 10% of the datapoints
Y_train_noisy[pts_to_flip] = 1 - Y_train_noisy[pts_to_flip] #we are corrupting the responses of these datapoints

draw_results(x1_train, x2_train, color=Y_train_noisy, plot_title='Training data with noise')

In [ ]:
#Performance of Logistic Regression on this corrupted dataset:
logimodel_train = LogisticRegression(penalty = None, max_iter = 1000)
logimodel_train.fit(X_train, Y_train_noisy)
print("Intercept:", logimodel_train.intercept_)
print("Coefficients:", logimodel_train.coef_)
#Prediction for the test data
Y_pred = logimodel_train.predict(X_test)
print(Y_pred)
accuracy_logimodel = np.mean(Y_test == Y_pred)
print(f"Accuracy of Logistic Regression on test set: {accuracy_logimodel}")

In [ ]:
#Logistic Regression with the x1*x2 feature
logimodel_train_feat = LogisticRegression(penalty = None, max_iter = 1000)
logimodel_train_feat.fit(X_train_feat, Y_train_noisy)
print("Intercept:", logimodel_train_feat.intercept_)
print("Coefficients:", logimodel_train_feat.coef_)
#Prediction for the test data
Y_pred_feat = logimodel_train_feat.predict(X_test_feat)
print(Y_pred_feat)
accuracy_logimodel_feat = np.mean(Y_test == Y_pred_feat)
print(f"Accuracy of Logistic Regression (with new feature x1*x2) on test set: {accuracy_logimodel_feat}")

In [ ]:
#Performance of k-NN on this corrupted data:
k_value = 8
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k_value)
knn.fit(X_train, Y_train_noisy)
Y_pred_knn = knn.predict(X_test)
accuracy_knn = np.mean(Y_test == Y_pred_knn)
print(f"Accuracy of k-NN Classification on test set: {accuracy_knn}")
#Contingency table (or confusion matrix)
print(pd.crosstab(Y_test, Y_pred_knn, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [ ]:
#Performance of Decision Tree on corrupted data:
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, Y_train_noisy)

# Predict on the test set
Y_pred_dt = dt.predict(X_test)

# Calculate the accuracy
accuracy_dt = np.mean(Y_test == Y_pred_dt)
print(f"Accuracy of Decision Tree Classification on test set: {accuracy_dt}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_dt, rownames=['Actual'], colnames=['Predicted'], margins=True))

# Plot the tree
from sklearn.tree import plot_tree
plt.figure(figsize=(20,10))
plot_tree(dt, feature_names=['x1', 'x2'],class_names = ('0','1'),  filled=True, rounded=True)
plt.show()

In [ ]:
#Performance of Random Forest:
rf = RandomForestClassifier(random_state=1, max_features = None)
rf.fit(X_train, Y_train_noisy)

# Predict on the test set
Y_pred_rf = rf.predict(X_test)

# Calculate the accuracy
accuracy_rf = np.mean(Y_test == Y_pred_rf)
print(f"Accuracy of Random Forest Classification on test set: {accuracy_rf}")

# Create the confusion matrix (contingency table)
print(pd.crosstab(Y_test, Y_pred_rf, rownames=['Actual'], colnames=['Predicted'], margins=True))